# theano Tutorial


## What’s Theano
A Python library for symbolic mathematics 
* Symbolic expression language/compiler 
* Tightly integrated with the Python ecosystem
* Fast C/CUDA back-end and transparent GPU acceleration

Developed at University of Montreal / Canada (January 2008)

## Basic Concepts
### General Steps of theano Applications
* import theano package
* Define theano _variables_
* Define _symbolic expressions_ using variables
* Compile a _function_ that can compute numeric values using expressions
* Execute that function on data

#### import theano package

In [ ]:
import theano
import theano.tensor as T
theano.__version__

In [ ]:
# Other libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Define theano variables
##### Constants

In [ ]:
Lr = T.constant(0.01, 'LearingRate', dtype='float64')
print (Lr)

##### Scalars (0-d)

In [ ]:
Lr = T.scalar(dtype='float64')
# or
Lr = T.dscalar('LearningRate')

Other scalar types:

|Usage      |Size/Type |
|-----------|----------|
|T.fscalar()|float32   |
|T.dscalar()|float64   |
|T.bscalar()|int8      |
|T.wscalar()|int16     |
|T.iscalar()|int32     |
|T.lscalar()|int64     |
|T.cscalar()|complex64 |
|T.zscalar()|complex128|

##### Vectors (1-d)

In [ ]:
X = T.vector('X', dtype='float32')
# or
Y = T.dvector('Y') # dtype='float64'

##### Matrices (2-d)

In [ ]:
W = T.matrix('W', dtype='float32')
# or
b = T.dmatrix('b') # dtype='float64'

#### Define symbolic expressions using variables
Example: $y = 2 * x + 5$

In [ ]:
a = T.constant(2)
x = T.scalar('x')
c = T.constant(5)

In [ ]:
y = a * x + c

#### Compile Function
Example: $f(x) = y = 2 * x + 5$

In [ ]:
fx = theano.function(inputs=[x], outputs=y)

#### Execute function
Example: $f(5)$

In [ ]:
# Now let's try


#### More on Symbolic Expressions

In [ ]:
from theano import pp


In [ ]:
pp(y)

In [ ]:
pp(fx.maker.fgraph.outputs[0])

In [ ]:
theano.printing.debugprint(y)

In [ ]:
theano.printing.debugprint(fx)

In [ ]:
theano.printing.pydotprint(y, outfile="y.png", var_with_name_simple=True)

![y](y.png)

In [ ]:
theano.printing.pydotprint(fx, outfile="fx.png", var_with_name_simple=True)

![fx](fx.png)

#### Vector Example
Example: $f(X,Y) = X + Y$, where $X$, and $Y$ are vectors.

In [ ]:
# Define Symbolic Variables
X = T.vector('X')
Y = T.vector('Y')

In [ ]:
# Define Symbolic Expression
sum = X+Y

In [ ]:
# Define Function f(X,Y) 
sumXY = theano.function(inputs=[X,Y], outputs=sum)

In [ ]:
# add [1,1,1] and [2,2,2]

#### Matrix Example
Example: $f(X) = X^T$ and $g(X,Y) = X Y$

In [ ]:
# Define Symbolic Variables
Xm = T.matrix('X')
Ym = T.matrix('Y')

In [ ]:
# Define Symbolic Expression
X_transpose = T.transpose(Xm)
XY_prod = T.dot(Xm,Ym)

In [ ]:
# Define Functions
Xt = theano.function([Xm], X_transpose)

In [ ]:
X = np.matrix([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
], dtype='float32')
print "X="; print X

Y = np.identity(3) * 5
print "Y="; print Y

In [ ]:
Xt(X)

In [ ]:
prodXY = theano.function([Xm,Ym], XY_prod)

In [ ]:
prodXY(X, Y)

#### Tensor Operations
* Data types (vector, matrix, tensor3, tensor4, etc.)
* Vector/Matrix (dot, transpose, ones,  zeros, etc.)
* Power (power, exp, exp2, etc.)
* Logarithm (log, log10, log2, log1p, etc.)
* Trigonometry (sin[h], cos[h], tan[h], etc.)
* Logic operators (and, or, xor, etc.) 
* Comparators (eq, lt, gt, ge, le, neq, etc.) 
* Bitwise operators (bitwise_and, bitwise_or, etc.)
* max, min, argmax, argmin, etc.
* square, sqrt, etc.
* ceil, floor, etc.

In [ ]:
logX = T.log(Xm)

In [ ]:
flX = theano.function([Xm], logX)

In [ ]:
flX(X)

## More Theano Concepts
### Default Input Values
Example: Consider following function definition in python:
<pre>
def fxy(x, y=1): 
</pre>
Let's mimic this with _theano_.

In [ ]:
from theano import In
# Define Symbolic Expression
x, y = T.dscalars('x', 'y')
z = x + y

# Define Functions
fxy = theano.function([x, In(y, value=1)], z)

In [ ]:
fxy(33)

In [ ]:
fxy(33,2)

### Input Variable Names
Example: Consider following function definition in python:
<pre>
def fxyw(x, y=1, weight=2): 
</pre>
Let's mimic this with _theano_.


In [ ]:
# Define Symbolic Expression
x, y, w = T.dscalars('x', 'y', 'w')
z = (x + y) * w

# Define Functions
fxyw = theano.function([x, In(y, value=1), 
                           In(w, value=2, name='weight')], z)


In [ ]:
fxyw(33)

In [ ]:
fxyw(33, 2)

In [ ]:
fxyw(33, 0, 1)

In [ ]:
fxyw(33, weight=1)

In [ ]:
fxyw(33, weight=1, y=0)

### Shared Variables 
Example: Create a shared variable to count number of _theano_ function calls.
<pre>
def fy(&lt;inputs&gt;):
    &lt;function body&gt;
    count = count + 1
</pre>

In [ ]:
# Define Symbolic Expression
a = T.constant(2)
x = T.scalar()
c = T.constant(5)
y = a * x + c 
z = y**2

# Define shared variable
count = theano.shared(0)

# Define Functions
fy = theano.function([x], y, updates=[(count, count+1)])
fz = theano.function([x], z, updates=[(count, count+1)])

In [ ]:
print count.get_value() # count = 0
print fy(2) 
print fz(2) # y^2
print count.get_value() # count = 2

In [ ]:
count.set_value(0)
print count.get_value() # count = 0

### Variable Substitution
Example:

In [ ]:
# Define variables
x = T.scalar()
y = 2 * x + 5 
z = y**2

# define function
fy2 = theano.function([x], z)

In [ ]:
# new variable. Same type as y.
n=T.scalar('n', dtype=y.dtype)

# new function.
# y in the calculation tree is replaced by n.
fyn = theano.function([n], z, givens=[(y, n)])

In [ ]:
print fy2(5)

In [ ]:
print fyn(5)

### Random Variables
Example: Create a uniform random variable.

In [ ]:
from theano.tensor.shared_randomstreams import RandomStreams
# Create a RNG instance, as a “shared variable”
rng = RandomStreams()

# Create a 'uniform' random variable
rv_u = rng.uniform()

In [ ]:
# create a theano function from the variable
fu = theano.function([], rv_u)

In [ ]:
for i in range(10):
    print fu()

In [ ]:
anArray = []
for i in range(1000):
    anArray.append(fu())

In [ ]:
plt.scatter(range(1000),anArray)

In [ ]:
# Seeding RNG
rng = RandomStreams(seed=1234)

In [ ]:
# Other distributions
rv_u = rng.binomial((3,3))
rv_n = rng.normal((2,2))
rv_i = rng.random_integers(high=1024)

In [ ]:
# Use RV as regular shared variable
y = rv_i ** 2

# define Theano function
gi = theano.function([], (rv_i, y))

In [ ]:
for i in range(10):
    print gi()

In [ ]:
# Create a RNG class
rng = RandomStreams()
# Create a 'uniform' random variable  
rv_u = rng.uniform() 

# create a theano function from the variable
gu = theano.function([], rv_u, no_default_updates=True)

In [ ]:
for i in range(5):
    print gu()

In [ ]:
# create output y from rv_u
y = 2 * rv_u - 2 * rv_u
fuy = theano.function([], y)

In [ ]:
for i in range(5):
    print fuy()

### Evaluating Expressions

In [ ]:
# Define Symbolic Variables
a = T.constant(2)
x = T.scalar()
c = T.constant(5)

# Define Symbolic Expression
y = a * x + c # Also y = 2 * x + 5

# Define Function f(x) = y
fx = theano.function(inputs=[x], outputs=y)

fx(2)

In [ ]:
# Execute/Calculate
print y.eval({x: 2})

## Examples
### [Logistic Regression](01-Theano_logistic.ipynb)
### [Multilayer Perceptron](02-Theano_MLP.ipynb)
### [Convolutional Neural Networks (LeNeT5)](03-Theano_lenet.ipynb)


## Installation Notes
### Installing theano on Windows
* WinPython 
    * Install WinPython
    * Includes theano 0.7 and other dependencies
    * Includes the g++ compiler
    * Add g++ compiler to your PATH
* Anaconda
    * Install Anaconda, or miniconda
    * Install theano, _conda install theano_
    * Install g++ compiler, _conda install m2w64-gcc_
* For GPU
    * Install latest drivers for GPU card
    * Install CUDA Development Kit
    * Install cuDNN if your GPU supports
    * Install Visual C++ compiler

### Installing theano on Linux
* Install/update python 
<pre>
$sudo apt-get install python-numpy python-scipy python-dev python-pip python-nose g++ libopenblas-dev git 
$sudo pip install Theano
</pre>

* For GPU
    * Install latest drivers for GPU card
    * Install CUDA Development Kit
    * Install cuDNN if your GPU supports
    * Add nvcc compiler to your PATH

### Using GPU with theano
* Environment variable
<pre>
export THEANO_FLAGS=“device=gpu0,floatX=float32”
python …
</pre>
or
<pre>THEANO_FLAGS=“device=gpu0,floatX=float32” python …</pre>

* Config File ~/.theanorc
<pre>
[global]
floatX = float32
device = gpu0 
</pre>

## Resources

* Theano Website: http://deeplearning.net/software/theano
* Theano Documentation: http://deeplearning.net/software/theano/#documentation
* Deep Learning with Theano: http://www.deeplearning.net/tutorial
* Theano Workshop: https://github.com/goodfeli/theano_exercises
* WinPython: http://winpython.sourceforge.net
* Anaconda: http://www.continuum.io/